In [56]:
from eodhd import APIClient
import requests 
import pandas as pd
import time
from tqdm import tqdm
import os


api = "64d77f6d3a60a5.24835840"

In [57]:
# First define the log function to gather the log information
def log(response,logfile, stock, offset,output_path=os.getcwd()):
    # Open or create the csv file
    if os.path.isfile(logfile): #If the log file exists, open it and allow for changes     
        log = open(logfile,'a')
    else: #If the log file does not exist, create it and make headers for the log variables
        log = open(logfile,'w')
        header = ['timestamp','status_code','length','Ticker','Offset']
        log.write(';'.join(header) + "\n") #Make the headers and jump to new line
        
    # Gather log information
    status_code = response.status_code #Status code from the request result
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())) #Local time
    length = len(response.text) #Length of the HTML-string
    
    # Open the log file and append the gathered log information
    with open(logfile,'a') as log:
        log.write(f'{timestamp};{status_code};{length};{stock};{offset}' + "\n") #Append the information and jump to new line

In [44]:
# Opening the csv file named NASDAQ 100, reading it and making it into a dataframe

# Get the current working directory
current_directory = os.getcwd()

# Construct the path to the CSV file
csv_file_path = os.path.join(current_directory, 'Data collection','Data','NASDAQ 100.csv')

# Read the CSV file and create a DataFrame
df_companies = pd.read_csv(csv_file_path, delimiter=';', encoding='utf-8')

# make df_companies into a dataframe
df_companies = pd.DataFrame(df_companies)

In [58]:
# Get all news

# Dates
start_date = '2020-01-01'
end_date = '2023-07-31'

# number of articles pr. stock
n_news = 1
stock_list = []
stock_list = df_companies['Symbol']
company_names = []
company_names = df_companies['Company']

logfile = 'log.csv'

def get_all_news(stocks, start_date, end_date, n_news, api_key, offset = 0):
    # create empty dataframe
    all_news = pd.DataFrame()
    for stock in tqdm(stocks, desc="Fetching News", unit="stock"):
        # url for the api call
        url = f'https://eodhistoricaldata.com/api/news?api_token={api_key}&s={stock}&limit={n_news}&offset={offset}&from={start_date}&to={end_date}'
        # get the json from the api call
        response = requests.get(url)
        news_json = response.json()
        
        # create dataframe from json
        df_news = pd.DataFrame.from_dict(news_json)
        # filter the dataframe
        df_news = df_news[['date','title','content','symbols','sentiment']]
        
        # a new column with the company name 
        df_news['company'] = company_names[stock_list == stock].values[0]
        
        # concat df_news onto all_news
        all_news = pd.concat([all_news, df_news], ignore_index = True)
        
        # Log the response
        log(response, logfile, stock, offset)
        
        # save the dataframe
        df_all_news.to_csv('df_all_news.csv', index=False)
    return all_news 

df_all_news = get_all_news(stock_list, start_date, end_date, n_news, api)


df_all_news



Fetching News: 100%|██████████| 101/101 [01:39<00:00,  1.01stock/s]


,date,title,content,symbols,sentiment,company
0,2023-07-31T23:54:16+00:00,"Asian stocks push higher on tech strength, Chi...",Investing.com -- Most Asian stocks rose on Tue...,"[0700.HK, 2007.HK, 9888.HK, 9988.HK, AAPL.US, ...","{'polarity': 0.992, 'neg': 0.066, 'neu': 0.794...",Apple Inc
1,2023-07-31T23:54:16+00:00,"Asian stocks push higher on tech strength, Chi...",Investing.com -- Most Asian stocks rose on Tue...,"[0700.HK, 2007.HK, 9888.HK, 9988.HK, AAPL.US, ...","{'polarity': 0.992, 'neg': 0.066, 'neu': 0.794...",Microsoft Corp
2,2023-07-31T23:54:16+00:00,"Asian stocks push higher on tech strength, Chi...",Investing.com -- Most Asian stocks rose on Tue...,"[0700.HK, 2007.HK, 9888.HK, 9988.HK, AAPL.US, ...","{'polarity': 0.992, 'neg': 0.066, 'neu': 0.794...",Amazon.com Inc
3,2023-07-31T21:01:39+00:00,The 2023 stock market rally got a lot healthie...,"July has come and gone for investors, bringing...","[AAPL.MX, AAPL.US, AAPL34.SA, ABEA.F, ABEA.XET...","{'polarity': 0.991, 'neg': 0.026, 'neu': 0.88,...",NVIDIA Corp
4,2023-07-31T23:54:16+00:00,"Asian stocks push higher on tech strength, Chi...",Investing.com -- Most Asian stocks rose on Tue...,"[0700.HK, 2007.HK, 9888.HK, 9988.HK, AAPL.US, ...","{'polarity': 0.992, 'neg': 0.066, 'neu': 0.794...",Meta Platforms Inc
...,...,...,...,...,...,...
96,2023-07-31T18:00:47+00:00,Jim Cramer’s 10 S&amp;P 500 Stock Picks for 2023,"In this article, we discuss Jim Cramer's 10 S&...","[A1MD34.SA, AMD.F, AMD.MX, AMD.US, AMD.XETRA, ...","{'polarity': 0.999, 'neg': 0.071, 'neu': 0.794...",Enphase Energy Inc
97,2023-07-30T12:03:00+00:00,Sirius XM Holdings' (NASDAQ:SIRI) Dividend Wil...,Sirius XM Holdings Inc. (NASDAQ:SIRI) will pay...,"[RDO.F, SIRI.US]","{'polarity': 0.999, 'neg': 0.015, 'neu': 0.822...",Sirius XM Holdings Inc
98,2023-07-31T02:00:50+00:00,16 Fastest Growing Blue-Collar Jobs,"In this article, we will look at 16 fastest-gr...","[DYH.F, HD.MX, HD.US, HDI.F, HDI.XETRA, HOME34...","{'polarity': 1, 'neg': 0.043, 'neu': 0.81, 'po...",Zoom Video Communications Inc
99,2023-07-30T11:01:07+00:00,"Is JD.com, Inc.'s (NASDAQ:JD) Recent Stock Per...",JD.com (NASDAQ:JD) has had a great run on the ...,"[013A.F, 9618.HK, JD.US, JDCMF.US, JDCO34.SA]","{'polarity': 0.999, 'neg': 0.02, 'neu': 0.806,...",JD.com Inc ADR


In [11]:
# save df_all_news to csv
df_all_news.to_csv('df_all_news.csv', index=False)